In [1]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

import  warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.
C:\Users\avani\AnacondaLatest\envs\MLBox\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\avani\AnacondaLatest\envs\MLBox\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\avani\AnacondaLatest\envs\MLBox\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\avani\AnacondaLatest\en

In [2]:
# reading and cleaning the train and test files
df = Reader(sep = ",").train_test_split(['train_bm.csv','test_bm.csv'] , 'Item_Outlet_Sales')


reading csv : train_bm.csv ...
cleaning data ...
CPU time: 0.15099406242370605 seconds

reading csv : test_bm.csv ...
cleaning data ...
CPU time: 5.038973569869995 seconds

> Number of common features : 11

gathering and crunching for train and test datasets ...
reindexing for train and test datasets ...
dropping training duplicates ...
dropping constant variables on training set ...

> Number of categorical features: 7
> Number of numerical features: 4
> Number of training samples : 8523
> Number of test samples : 5681

> Top sparse features (% missing values on train set):
Outlet_Size    28.3
Item_Weight    17.2
dtype: float64

> Task : regression
count     8523.000000
mean      2181.288914
std       1706.499616
min         33.290000
25%        834.247400
50%       1794.331000
75%       3101.296400
max      13086.964800
Name: Item_Outlet_Sales, dtype: float64


In [3]:
df.keys()

dict_keys(['train', 'test', 'target'])

In [4]:
print('Train data shape -' , df['train'].shape)
print('Test data shape - ', df['test'].shape)

Train data shape - (8523, 11)
Test data shape -  (5681, 11)


In [5]:
df['train'].isnull().sum()

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Type                       0
Item_Visibility                 0
Item_Weight                  1463
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  2410
Outlet_Type                     0
dtype: int64

In [6]:
df['test'].isnull().sum()

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Type                       0
Item_Visibility                 0
Item_Weight                   976
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  1606
Outlet_Type                     0
dtype: int64

In [7]:
# optimisation
space = {'ne__numerical_strategy': {'space': ['median','mean','mode']},
         'ne__categorical_strategy' : {'space': ['mode']} , 
         
         'ce__strategy': {'search':'choice', 'space': ['label_encoding','entity_embedding']},
         'fs__strategy': {'search': 'choice', 'space': ['variance', 'rf_feature_importance']},
         'fs__threshold': {'search': 'uniform', 'space': [0.01, 0.3]},
         
         #model building strategy
         'est__strategy' : {'space': ['RandomForest']},
         'est__max_depth' : {'search':'choice', 'space': [3,4,8,9]},
         'est__n_estimators': {'search':'choice' , 'space': [250,400,500,1000]}         
        }

In [8]:
best = Optimiser(scoring='r2',n_folds = 5).optimise(space,df,10)

##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': 'mode'}
>>> CA ENCODER :{'strategy': 'entity_embedding'}    
>>> FEATURE SELECTOR :{'strategy': 'rf_feature_importance', 'threshold': 0.2160306535156606}
>>> ESTIMATOR :{'strategy': 'RandomForest', 'max_depth': 9, 'n_estimators': 500, 'bootstrap': True, 'criterion': 'mse', 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_jobs': -1, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}
  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]WARNING:tensorflow:From C:\Users\avani\AnacondaLatest\envs\MLBox\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.g

CPU time: 41.5283567905426 seconds                                            
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': 'mode'}
>>> CA ENCODER :{'strategy': 'label_encoding'}                                
>>> FEATURE SELECTOR :{'strategy': 'variance', 'threshold': 0.054030111395777417}
>>> ESTIMATOR :{'strategy': 'RandomForest', 'max_depth': 4, 'n_estimators': 250, 'bootstrap': True, 'criterion': 'mse', 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_jobs': -1, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}
MEAN SCORE : r2 = 0.5272689322849053                                          
VARIANCE : 0.009779325768345415 (fold 1 = 0.5286655451855939, fold 2 = 0

In [9]:
best

{'ce__strategy': 'label_encoding',
 'est__max_depth': 8,
 'est__n_estimators': 500,
 'est__strategy': 'RandomForest',
 'fs__strategy': 'rf_feature_importance',
 'fs__threshold': 0.16544538179164675,
 'ne__categorical_strategy': 'mode',
 'ne__numerical_strategy': 'mean'}

In [10]:
# predictions on Test Data
Predictor().fit_predict(best,df)


fitting the pipeline ...
CPU time: 6.301243305206299 seconds

predicting...
CPU time: 0.3739910125732422 seconds

> Overview on predictions : 

   Item_Outlet_Sales_predicted
0                  1694.197890
1                  1435.819320
2                   604.026889
3                  2600.835689
4                  5703.266577
5                  1830.101099
6                   846.531011
7                  2241.469742
8                  1586.501548
9                  2992.014398

dumping predictions into directory : save ...
